In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Change root directory
dirPath = r'C:\Users\Andreas\Desktop\Thesis\Data'
os.chdir(dirPath)
print('Current working directory: {0}'.format(os.getcwd()))

Current working directory: C:\Users\Andreas\Desktop\Thesis\Data


In [8]:
def read_json(dirPath = dirPath):
    labels = pd.DataFrame()
    for file in os.listdir(dirPath):
        if 'project' in file:
            json_table = pd.read_json(dirPath+'\\'+file)
            json_table = json_table[['csv', 'label']]
    
    for _, row in json_table.iterrows():
        subjectName = row.csv.split('/')[5]
        exerciseName = row.csv.split('/')[6]
        
        rep = 0
        for item in row.label:
                start = item['start']
                end = item['end']
                label = item['timeserieslabels'][0]

                if label == 'idle':
                    dict = {'subject':subjectName,'exercise':exerciseName,'start':start,'end':end,'label':label, 'rep': 0}
                else:
                    rep +=1
                    dict = {'subject':subjectName,'exercise':exerciseName,'start':start,'end':end,'label':label, 'rep': rep}
                     
                labels = pd.concat([labels, pd.DataFrame(dict,index=[0])],ignore_index=True)
    
    labels = labels.drop(labels[labels.label == 'idle'].index)
    labels = labels.drop(labels[labels.label == 'TBD'].index)

    dt = labels.end - labels.start
    print(dt.describe())
    labels = labels.loc[dt>dt.quantile(.05)]
    labels = labels.loc[dt<dt.quantile(.95)]

    dt = labels.end - labels.start
    print(dt.describe())

    return labels

In [9]:
labels = read_json()

count    1059.000000
mean        2.582049
std         0.569521
min         1.140000
25%         2.200000
50%         2.510000
75%         2.920000
max         5.520000
dtype: float64
count    952.000000
mean       2.563487
std        0.436582
min        1.800000
25%        2.230000
50%        2.515000
75%        2.860000
max        3.600000
dtype: float64


In [5]:
labels = labels.replace('correct', 'Correct repetition')

labels = labels.replace('knee_valgus', 'Knees caving inwards')
labels = labels.replace('discreet_motion', 'Hips rising before torso')
labels = labels.replace('spine_flexion', 'Back rounding')
labels = labels.replace('spine_extension', 'Back arching')

labels = labels.replace('pelvis_tilt', 'Back arching')
labels = labels.replace('incomplete', 'Insufficient ROM')
labels = labels.replace('knee_flexion', 'Feet too close')
labels = labels.replace('knee_extension', 'Feet too far')

labels = labels.replace('knee_touch', 'Knee touching the ground')
labels = labels.replace('short_step', 'Stride too short')
labels = labels.replace('forward_centre', 'Leaning forward')
labels = labels.replace('imbalance', 'Feet on the same line')

In [6]:
def label_csv(dirPath=dirPath, sample = 'all', exercise='all'):
    root = dirPath+'\\'+'Synchronized'
    total_size = 0
    for samplefolder in os.listdir(root):
        if sample in samplefolder or sample == 'all':
            sample_group = labels.loc[labels.subject == samplefolder]

            for exercisefolder in os.listdir(root+'\\'+samplefolder):
                if exercise in exercisefolder or exercise == 'all':
                    exercise_group = sample_group.loc[sample_group.exercise == exercisefolder]

                    if exercise_group.empty:
                        print(samplefolder+exercisefolder+' is not annotated')
                    else:
                        for file in os.listdir(root+'\\'+samplefolder+'\\'+exercisefolder):
                            if '.csv' in file:
                                    csv_path = root+'\\'+samplefolder+'\\'+exercisefolder+'\\'+file

                                    data = pd.read_csv(csv_path, delimiter=',', encoding = 'utf-8')

                                    file_size = len(data)
                                    total_size = total_size + file_size
                                    data.insert(len(data.columns),'label','Unassigned')
                                    data.insert(len(data.columns),'rep',0)
                                
                                    for _, row in exercise_group.iterrows():
                                        data.loc[(data.t > row.start) & (data.t < row.end), 'label'] = row.label
                                        data.loc[(data.t > row.start) & (data.t < row.end), 'rep'] = row.rep

                                    new_root = dirPath+'\\'+'Labelled'+'\\'+samplefolder+'\\'+exercisefolder

                                    if not os.path.exists(new_root):
                                        os.makedirs(new_root)

                                    data.drop(data[data.label == 'Unassigned'].index, inplace = True)
                                    data.to_csv(new_root+'\\'+file, sep=',', index=False)

    print(total_size)                           

In [7]:
label_csv()

4949056
